In [ ]:
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['animation.html'] = 'jshtml'
mpl.rcParams['animation.embed_limit'] = 200.0

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import happi

# -------- helpers
def to_array(x):
    """Coerce happi outputs (lists/object arrays) to a numeric numpy array."""
    a = np.asarray(x)
    if a.dtype == object:
        try:
            a = np.array(x, dtype=float)
        except Exception:
            a = np.array([np.asarray(e, dtype=float) for e in x], dtype=float)
    return a

def safe_axis(diag, name, n_fallback):
    try:
        v = to_array(diag.getAxis(name))
        if v.ndim == 0:  # scalar? fall back
            raise ValueError
        return v
    except Exception:
        return np.arange(n_fallback)

# -------- load data
# path = "../work_dir/tuto1"
path = "../work_dir/off/2D/t1"
field_component = "Bz"

S = happi.Open(path, verbose=False)
diag = S.Field(0, field_component)

ts = np.array(diag.getTimesteps()).tolist()
if len(ts) == 0:
    raise RuntimeError("No timesteps found in this Field diagnostic.")

# Subsample to ~100 frames for snappy playback
step = max(1, len(ts) // 100)
ts = ts[::step]

# First frame
frame0 = to_array(diag.getData(timestep=ts[0]))
if frame0 is None:
    raise RuntimeError("getData() returned None. Check the path/diagnostic.")

# If >2D (rare), pick the first slice
if frame0.ndim > 2:
    frame0 = frame0[0]

fig, ax = plt.subplots(figsize=(6, 4))

if frame0.ndim == 2:
    ny, nx = frame0.shape
    x = safe_axis(diag, "x", nx)
    y = safe_axis(diag, "y", ny)
    extent = [x.min(), x.max(), y.min(), y.max()]

    im = ax.imshow(frame0, origin="lower", aspect="auto", extent=extent, interpolation="nearest")
    cbar = fig.colorbar(im, ax=ax); cbar.set_label(field_component)

    def update(i):
        arr = to_array(diag.getData(timestep=ts[i]))
        if arr.ndim > 2:
            arr = arr[0]
        im.set_data(arr)
        return (im,)

elif frame0.ndim == 1:
    nx = frame0.shape[0]
    x = safe_axis(diag, "x", nx)
    (ln,) = ax.plot(x, frame0)
    ax.set_xlabel("x"); ax.set_ylabel(field_component)

    def update(i):
        ydata = to_array(diag.getData(timestep=ts[i]))
        if ydata.ndim > 1:  # collapse if weird shape sneaks in
            ydata = ydata.ravel()
        ln.set_ydata(ydata)
        return (ln,)

else:
    raise ValueError(f"Unsupported data dim: {frame0.ndim}. Expected 1-D or 2-D.")

ax.set_title(field_component+f" (frames: {len(ts)})")
fig.tight_layout()

ani = FuncAnimation(fig, update, frames=len(ts), interval=50, blit=False)
display(HTML(ani.to_jshtml()))
plt.close(fig)

In [ ]:
path = "../work_dir/tuto1"

In [ ]:
# open Smilei results
S = happi.Open(path, verbose=False)

In [ ]:
diag = S.Scalar('Uelm')
diag.plot()

In [ ]:
happi.multiPlot(
    S.Scalar('Utot', label="Total energy"),
    S.Scalar('Ubal', label="Balance")
    )

In [ ]:
diag = S.Field(0, "Ey")
ani=diag.slide(vsym=1)
HTML(ani.to_jshtml())             # explicitly display it

In [ ]:
S.Field(0, "Ey**2+Ez**2", average={"x":[0,7],"y":100}).plot()

In [ ]:
laser = S.namelist.Laser[0]
from numpy import array, arange
tstop = S.namelist.Main.simulation_time # simulation final time
tstep = S.namelist.Main.timestep        # simulation timestep
times = arange(0., tstop, tstep)
laser_profile = array([laser.time_envelope(t) for t in times])

%matplotlib inline

fig0 = figure()
gs = GridSpec(2, 3)
ax0 = subplot(gs[:,:])
ax0.plot( times+5, laser_profile**2 / 2 )

print(times)